In [1]:
import os
import sys

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from src.models import ModelXtoCResNet
from src.preprocessing.Derm7pt import preprocessing_Derm7pt
from src.utils import *
from src.training import run_epoch_x_to_c

In [2]:
concept_labels, train_loader, val_loader, test_loader = preprocessing_Derm7pt(training=True, class_concepts=True, verbose=True)

Found 34 classes.
Found labels for 2022 images.
Generated one-hot matrix with shape: (2022, 34)
Total number of concept columns: 28
Found 2013 images.
Processing in 32 batches of size 64 (for progress reporting)...


Processing batches: 100%|██████████| 32/32 [00:13<00:00,  2.30it/s]



Finished processing.
Successfully transformed: 2013 images.
Labels shape: (2013, 34)
Concepts shape: (2013, 28)
Image tensors length: 2013
Dataset initialized with 826 pre-sorted items.
Dataset initialized with 406 pre-sorted items.
Dataset initialized with 790 pre-sorted items.


# Training Implementation

In [3]:
from src.utils import find_class_imbalance
from src.config import DERM7PT_CONFIG as config_dict
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
N_TRIMMED_CONCEPTS = config_dict['N_TRIMMED_CONCEPTS']
N_CLASSES = config_dict['N_CLASSES']

**Find device to run model on (CPU or GPU).**

In [5]:
device = torch.device("cuda" if torch.cuda.is_available()
                    else "mps" if torch.backends.mps.is_available()
                    else "cpu")
print(f"Using device: {device}")

Using device: mps


**Instantiate the model.**

In [6]:
model = ModelXtoCResNet(pretrained=True,
                freeze=True,
                n_concepts=N_TRIMMED_CONCEPTS)

model = model.to(device)
print("Model Instantiated (X -> C)")

/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model Instantiated (X -> C)


### Loss
We use weighted loss.

`BCEWithLogitsLoss()` performs 2 steps:
1. $\sigma(x)$
    - Applies the sigmoid function to the logits to get probabilities.
2. $\text{BCE}(\sigma(x), y) = y \cdot \text{log}(\sigma(x)) + (1-y) \cdot (1-\text{log}(\sigma(x)))$
    - Compute binary cross-entropy between output probabilities ($\sigma(x)$) and ground truths ($y$)

In [7]:
use_weighted_loss = True # Set to False for simple unweighted loss

if use_weighted_loss:
    concept_weights = find_class_imbalance(concept_labels)
    attr_criterion = [nn.BCEWithLogitsLoss(weight=torch.tensor([ratio], device=device, dtype=torch.float))
                    for ratio in concept_weights]
else:
    attr_criterion = [nn.BCEWithLogitsLoss() for _ in range(N_TRIMMED_CONCEPTS)]

### Optimiser
Use same settings as used in CBM repo.

In [8]:
lr = 0.01
weight_decay = 0.00004 # same as lambda in L2-regularisation

optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                    lr=lr,
                    momentum=0.9,
                    weight_decay=weight_decay)

# scheduler_step = n -> decrease the LR every n epochs
scheduler_step = 1000
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=0.1)

print("Optimizer and Scheduler Ready")

Optimizer and Scheduler Ready


### Training and Validation Loops

In [9]:
epochs = 50
log_interval = 50

best_val_acc = 0.0

In [10]:
for epoch in range(epochs):
    print(f"--- Epoch {epoch+1}/{epochs} ---")

    # Train
    train_loss, train_acc = run_epoch_x_to_c(model, train_loader, attr_criterion, optimizer, is_training=True, use_aux=True, n_concepts=N_TRIMMED_CONCEPTS, device=device, verbose=True)
    print(f'Epoch {epoch+1} Train Summary | Loss: {train_loss:.4f} | Acc: {train_acc:.3f}')

    # Validate
    if val_loader:
        with torch.no_grad():
            val_loss, val_acc = run_epoch_x_to_c(model, val_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device, verbose=True)

        print(f'Epoch {epoch+1} Val Summary   | Loss: {val_loss:.4f} | Acc: {val_acc:.3f}')

        # Save best model based on validation accuracy
        if val_acc > best_val_acc:
            print(f"Validation accuracy improved ({best_val_acc:.3f} -> {val_acc:.3f}). Saving model...")
            best_val_acc = val_acc
            torch.save(model, 'x_to_c_best_model.pth')
            print("Model saved to x_to_c_best_model.pth")

    # Scheduler step
    scheduler.step()
    print(f"Current LR: {optimizer.param_groups[0]['lr']}")

--- Epoch 1/50 ---


Epoch 1 Train Summary | Loss: 3.3447 | Acc: 85.204


Epoch 1 Val Summary   | Loss: 0.5120 | Acc: 91.397
Validation accuracy improved (0.000 -> 91.397). Saving model...
Model saved to x_to_c_best_model.pth
Current LR: 0.01
--- Epoch 2/50 ---


Epoch 2 Train Summary | Loss: 0.4746 | Acc: 92.191


Epoch 2 Val Summary   | Loss: 0.4517 | Acc: 91.327
Current LR: 0.01
--- Epoch 3/50 ---


...

Epoch 5 Val Summary   | Loss: 0.4137 | Acc: 91.529
Validation accuracy improved (91.502 -> 91.529). Saving model...
Model saved to x_to_c_best_model.pth
Current LR: 0.01
--- Epoch 6/50 ---


Epoch 6 Train Summary | Loss: 0.3753 | Acc: 92.511


Epoch 6 Val Summary   | Loss: 0.4127 | Acc: 91.643
Validation accuracy improved (91.529 -> 91.643). Saving model...
Model saved to x_to_c_best_model.pth
Current LR: 0.01
--- Epoch 7/50 ---


Epoch 7 Train Summary | Loss: 0.3701 | Acc: 92.498


RuntimeError: DataLoader worker (pid 7325) is killed by signal: Interrupt: 2. 

In [11]:
if test_loader:
    with torch.no_grad():

        test_loss, test_acc, outputs = run_epoch_x_to_c(
            model, test_loader, attr_criterion, optimizer=None, n_concepts=N_TRIMMED_CONCEPTS, device=device,
            return_outputs='sigmoid', verbose=True
        )

# print(f"Shuffled labels shape: {shuffled_img_labels.shape}")
print(f'Best Model Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

RuntimeError: Broken pipe